In [1]:
import findspark 
findspark.init()
import pyspark

In [2]:
import sparkmonitor
!jupyter nbextension install sparkmonitor --py --user --symlink 
!jupyter nbextension enable sparkmonitor --py --user            
!jupyter serverextension enable --py --user sparkmonitor
!ipython profile create && echo "c.InteractiveShellApp.extensions.append('sparkmonitor.kernelextension')" >>  $(ipython profile locate default)/ipython_kernel_config.pyimport sparkmonitor
!jupyter nbextension install sparkmonitor --py --user --symlink 
!jupyter nbextension enable sparkmonitor --py --user            
!jupyter serverextension enable --py --user sparkmonitor
!ipython profile create && echo "c.InteractiveShellApp.extensions.append('sparkmonitor.kernelextension')" >>  $(ipython profile locate default)/ipython_kernel_config.py

Installing D:\Anaconda\lib\site-packages\sparkmonitor\static -> sparkmonitor
- Validating: ok

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable sparkmonitor --user --py
    
Enabling notebook extension sparkmonitor/module...
      - Validating: ok
Enabling: sparkmonitor.serverextension
- Writing config: C:\Users\aserg\.jupyter
    - Validating...
      sparkmonitor.serverextension  ok
Installing D:\Anaconda\lib\site-packages\sparkmonitor\static -> sparkmonitor
- Validating: ok

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable sparkmonitor --user --py
    
Enabling notebook extension sparkmonitor/module...
      - Validating: ok
Enabling: sparkmonitor.serverextension
- Writing config: C:\Users\aserg\.jupyter
    - Validating...
      sparkmonitor.serverextension  ok


In [7]:
from pyspark.conf import SparkConf
conf = SparkConf()
conf.setAppName('nlpproject')
conf.setMaster('local[*]')
from pyspark.context import SparkContext
sc=SparkContext.getOrCreate(conf=conf) #Start the spark context
print(conf.toDebugString())

spark.app.name=nlpproject
spark.master=local[*]
spark.submit.deployMode=client
spark.ui.showConsoleProgress=true


In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlpproject').getOrCreate()

In [ ]:
import pandas as pd
data1 = pd.read_csv('DMF/ssdm1',sep='\s+', engine='python')

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [ ]:
data1.head()